In [1]:
import numpy as np
from pprint import pprint

import os 
from pathlib import Path

dir_path = str(Path(os.getcwd()).parent.absolute())
print(dir_path)

import sys
sys.path.append(dir_path)
if 'pyDWSIMopt.sim_opt' in sys.modules:  # Is the module in the register?
    del sys.modules['pyDWSIMopt.sim_opt']  # If so, remove it.
    del SimulationOptimization
from pyDWSIMopt.sim_opt import SimulationOptimization

c:\Users\lfsfr\Desktop\pyDWSIMopt


In [2]:
# Getting DWSIM path from system path
for k,v in enumerate(os.environ['path'].split(';')):
    if v.find('\DWSIM')>-1:
        path2dwsim = os.path.join(v, '')
if path2dwsim == None:
    path2dwsim = "C:\\Users\\lfsfr\\AppData\\Local\\DWSIM7\\"
print(path2dwsim)

# Loading DWSIM simulation into Python (Simulation object)
sim_smr = SimulationOptimization(dof=np.array([]), path2sim= os.path.join(dir_path, "examples\\SMR_LNG\\SMR.dwxmz"), 
                     path2dwsim = path2dwsim)
sim_smr.savepath = os.path.join(dir_path, "examples\\SMR_LNG\\SMR2.dwxmz")
sim_smr.Add_refs()

# Instanciate automation manager object
from DWSIM.Automation import Automation2
if ('interf' not in locals()):    # create automation manager
    interf = Automation2()

# Connect simulation in sim.path2sim
sim_smr.Connect(interf)

C:\Users\lfsfr\AppData\Local\DWSIM7\
<class 'Exception'>
More refs
added refs
Simulation was loaded successfully


In [4]:
# Assign DoF:
from pyDWSIMopt.py2dwsim import * 
assignDoF( ["MR-1", "CompoundMassFlow", "Nitrogen", "kg/s"], sim_smr )
assignDoF( ["MR-1", "CompoundMassFlow", "Methane", "kg/s"], sim_smr )
assignDoF( ["MR-1", "CompoundMassFlow", "Ethane", "kg/s"], sim_smr )
assignDoF( ["MR-1", "CompoundMassFlow", "Propane", "kg/s"], sim_smr )
assignDoF( ["MR-1", "CompoundMassFlow", "Isopentane", "Pa"], sim_smr )
assignDoF( ["VALV-01", "OutletPressure", "Nitrogen", "Pa"], sim_smr )
assignDoF( ["COMP-1", "OutletPressure", "Nitrogen", "Pa"], sim_smr )
assignDoF( ["COOL-08", "OutletTemperature", "Nitrogen", "K"], sim_smr )

# Assign F and G the same way


Material Stream
['DWSIM', 'Thermodynamics', 'Streams', 'MaterialStream']
Material Stream
['DWSIM', 'Thermodynamics', 'Streams', 'MaterialStream']
Material Stream
['DWSIM', 'Thermodynamics', 'Streams', 'MaterialStream']
Material Stream
['DWSIM', 'Thermodynamics', 'Streams', 'MaterialStream']
Material Stream
['DWSIM', 'Thermodynamics', 'Streams', 'MaterialStream']
Valve
['DWSIM', 'UnitOperations', 'UnitOperations', 'Valve']
Compressor
['DWSIM', 'UnitOperations', 'UnitOperations', 'Compressor']
Cooler
['DWSIM', 'UnitOperations', 'UnitOperations', 'Cooler']


In [6]:
# Initial simulation optimization setup
# Initial guess of optimization
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -60+273.15] )

# Testing for simulation at x0
sim_smr.calculate_optProblem(1.0*x0)
print(sim_smr.x_val,
      sim_smr.f_val,
      sim_smr.g_val)

# Test saving simulation at x0 in 'savepath'
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.savepath,True)

# Inspecting simulation object
pprint(vars(sim_smr))

f = [], g = [] at x = [6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.13150000e+02]
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.13150000e+02] [] []
{'dof': array([[<function toDwsim.<locals>.<lambda> at 0x0000019C291B9CA0>,
        'MR-1', 'CompoundMassFlow', 'Nitrogen', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x0000019C5D014DC0>,
        'MR-1', 'CompoundMassFlow', 'Methane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x0000019C61C1F0D0>,
        'MR-1', 'CompoundMassFlow', 'Ethane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x0000019C61C1F1F0>,
        'MR-1', 'CompoundMassFlow', 'Propane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x0000019C61C1F040>,
        'MR-1', 'CompoundMassFlow', 'Isopentane', 'Pa'],
       [<function toDwsim.<locals>.<lambda> at 0x0000019C61C1F280>,
        'VALV-01', 'OutletPre

In [7]:
# Setup for optimization
# convergence tolerances
xtol=0.01
ftol=0.01
maxiter=5 # +- 20 seconds per iteration

# decision variables bounds
bounds_raw = np.array( [0.5*np.asarray(x0), 1.5*np.asarray(x0)] )   # 50 % around base case
bounds_raw[0][-1] = 153     # precool temperature low limit manually
bounds_raw[1][-1] = 253     # precool temperature upper limit manually

# regularizer calculation
regularizer = np.zeros(x0.size)
import math
for i in range(len(regularizer)):
    regularizer[i] = 10**(-1*math.floor(math.log(x0[i],10))) # regularizer for magnitude order of 1e0

# bounds regularized
bounds_reg = regularizer*bounds_raw
bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])

# objective and constraints lambda definitions
f = lambda x: sim_smr.calculate_optProblem(np.asarray(x)/regularizer)[0:sim_smr.n_f]
g = lambda x: sim_smr.calculate_optProblem(np.asarray(x)/regularizer)[sim_smr.n_f:(sim_smr.n_f+sim_smr.n_g)]
nonlinear_constraint = optimize.NonlinearConstraint(g, -np.inf, 0, jac='2-point', hess=optimize.BFGS())

NameError: name 'optimize' is not defined

In [ ]:
# Global optimization with PSO
from sko.PSO import PSO

# f_pen = lambda x: fpen_barrier(sim_smr,x/regularizer)
result_pso = PSO(func= lambda x: sim_smr.fpen_barrier(x/regularizer), n_dim=sim_smr.n_dof, pop=2*sim_smr.n_dof, max_iter=40, lb=bounds_reg[0], ub=bounds_reg[1], verbose=True)
result_pso.record_mode = True
if sim_smr.n_f > 1:
    print("Multi-objective optimization not supported (yet)")
elif sim_smr.n_f < 1:
    print("Invalid number of objective functions")
else:
    print("Starting global optimization")
    result_pso.run()